# Tarea 4
0226594 || Sara Carolina Gómez Delgado

### Utils

In [44]:
import matplotlib
from sklearn.ensemble import RandomForestClassifier
import pandas
from bs4 import BeautifulSoup
import numpy as np
import scipy
import nltk 
from nltk.tokenize import TweetTokenizer

In [4]:
def get_texts_from_file(path_corpus, path_truth):
    tr_txt = []
    tr_y = []
    with open(path_corpus, "r",encoding="utf8") as f_corpus, open(path_truth, "r",encoding="utf8") as f_truth:
        for tuit in f_corpus:
            tr_txt += [tuit]
        for label in f_truth:
            tr_y += [label] 
    return tr_txt, tr_y

## 1. Modelos de Lenguaje y Evaluación

#### 1.1 Preprocesamiento

_<kbd>Instrucción</kbd>_

Preprocese todos los tuits de agresividad (positivos y negativos) según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de < s > y </ s > según usted considere (e.g., al
inicio y final de cada tuit). Defina su vocabulario y enmascare con < unk > toda palabra
que no esté en su vocabulario.


_<kbd>Comentario</kbd>_

En esta sección se preprocesaron los tweets y se generó un corpus donde sólo se consideraron palabras (no emojis, no signos de puntuación) y se eligió un vocabulario tomando las palabras únicas en el training set, después se ordenaron según su frecuencia (mayor a menor) y se tomaron sólo las palabras cuya frecuencia fue mayor a 50 para formar el vocabulario final. Me parece importante resaltar, como principal conclusión, la importancia de descartar palabras que aparecen poco, ya que pueden llegar a afectar fuertemente al intentar aplicar un modelo de lenguaje.

_Leer tweets_

In [5]:
tr_txt, tr_y = get_texts_from_file("./mex_train.txt", "./mex_train_labels.txt")

_Pre-procesar tweets_

In [73]:
# Create TweetTokenizer instance
tokenizer = TweetTokenizer()

In [74]:
# <s> at the beggining of a tweet and </s> at the end of this tweet
corpus_palabras = []
for doc in tr_txt:
    x = "<s>" + doc + "</s>"
    corpus_palabras += tokenizer.tokenize(x) 

In [104]:
processed = []
for word in corpus_palabras:
    if(np.char.isalpha(word) or word == "<s>" or word == "</s>"):
            processed.append(word)
print(processed[:10])

['<s>', 'lo', 'peor', 'de', 'todo', 'es', 'que', 'no', 'me', 'dan']


In [91]:
fdist = nltk.FreqDist(processed) # frecuencia de cada palabra

def sortFreqDict(freqdict):
    aux = [(freqdict[key], key) for key in freqdict] #lista de pares ordenada (más frecuente a menos)
    aux.sort()
    aux.reverse()
    return aux #regresa el objeto ordenado en reversa (más frecuentes a menos frecuentes)

V = sortFreqDict(fdist)

In [103]:
# if freq < 50, ignore it (don't let it be part of vocab)
vocab = []
for item in V:
    if item[0] > 50:
        vocab.append(item[1])
vocab[:10]

['<s>', '</s>', 'que', 'de', 'a', 'la', 'y', 'no', 'me', 'el']

In [98]:
for i,word in enumerate(processed):
    if word not in vocab:
        processed[i] = "<unk>"
print(processed)

['<s>', 'lo', '<unk>', 'de', 'todo', 'es', 'que', 'no', 'me', '<unk>', 'por', 'un', '<unk>', 'y', '<unk>', '<unk>', 'estoy', 'hasta', 'la', 'verga', 'de', '<unk>', '</s>', '<s>', 'a', 'la', '<unk>', 'no', '<unk>', '<unk>', 'putos', '<unk>', '<unk>', 'me', '<unk>', 'que', '<unk>', '<unk>', 'no', 'me', '<unk>', '<unk>', 'en', '<unk>', '</s>', '<s>', '<unk>', 'que', 'lo', 'más', '<unk>', '<unk>', 'que', 'lo', '<unk>', 'a', '<unk>', '<unk>', 'mi', '<unk>', '<unk>', 'la', 'madre', 'a', 'ese', '<unk>', 'joto', '</s>', '<s>', 'el', 'marica', 'de', 'mi', '<unk>', 'me', 'tiene', '<unk>', 'de', 'todo', 'así', 'uno', 'no', 'puede', '<unk>', 'la', '<unk>', 'de', 'su', '<unk>', '</s>', '<s>', 'mujer', '<unk>', 'pinche', '<unk>', '<unk>', 'esta', 'que', 'se', '<unk>', 'la', 'loca', '</s>', '<s>', 'putos', 'no', 'tienen', 'madre', '<unk>', '<unk>', '<unk>', 'como', 'es', '<unk>', 'mejor', '<unk>', '</s>', '<s>', '<unk>', 'si', '<unk>', '<unk>', 'de', '<unk>', 'pero', '<unk>', 'y', '<unk>', '<unk>', '

#### 1.2 Entrenamiento (unigrama, bigrama y trigrama)

_<kbd>Instrucción</kbd>_

Entrene tres modelos de lenguaje sobre todos los tuis: $P_{unigramas}(w^n_1), P_{bigramas}(w^n_1), P{trigramas}(w^n_1).$ Para cada uno proporcione  una interfaz (función) sencilla para $P{n-grama}(w^n_1)$ y $Pn-grama(w^n_1 | w^{n-1}_{n-N+1})$. Los modelos modelos deben tener una estrategia común para lidiar con
secuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turing
discounting. Muestre un par de ejemplos de como funciona, al menos uno con una
palabra fuera del vocabulario.

### 1.3 Construcción de Modelo Interpolado

_<kbd>Instrucción</kbd>_

Construya un modelo interpolado con valores $\lambda$ fijos:

$$\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1P(w_n|w_{n-2}w_{n-1}) + \lambda_2P(w_n|w_{n-1}) + \lambda_3P(w_n)$$

Para ello experimente con el modelo en particiones estratificadas de 80%, 10% y 10% para
entrenar (train), ajuste de parámetros (val) y prueba (test) respectivamente. Muestre como
bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Para esto puede explorar algunos valores ⃗λ y elija el mejor, i.e., [1/3, 1/3, 1/3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1]
y [.1, .4, .5].

## 2. Generación de Texto

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores λ:

$$\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1P(w_n|w_{n-2}w_{n-1}) + \lambda_2P(w_n|w_{n-1}) + \lambda_3P(w_n)$$

### 2.1 Función "tuitear"

_<kbd>Instrucción</kbd>_

Haga una función "tuitear" con base en su modelo de lenguaje P̂ del último punto.
El modelo deberá poder parar automáticamente cuando genere el símbolo de terminación
de tuit al final (e.g., "</s>"), o 50 palabras. Proponga algo para que en los últimos tokens
sea más probable generar el token "</s>". Muestre al menos cinco ejemplos.

### 2.2 Entrenar modelo de lenguaje AMLO

_<kbd>Instrucción</kbd>_

Use la intuición que ha ganado en esta tarea y los datos de las mañaneras para
entrenar un modelo de lenguaje AMLO. Haga una un función "dar_conferencia()". Generé
un discurso de 300 palabras y detenga al modelo de forma abrupta.

### 2.3 Experimentando con dos frases

_<kbd>Instrucción</kbd>_

Calcule el estimado de cada uno sus modelos de lenguaje (el de tuits y el de amlo)
para las frases: "sino gano me voy a la chingada", "ya se va a acabar la corrupción".

### 2.4 Permutaciones

_<kbd>Instrucción</kbd>_

Para cada oración del punto anterior, haga todas las permutaciones posibles.
Calcule su probabilidad a cada nueva frase y muestre el top 3 mas probable y el top 3
menos probable (para ambos modelos de lenguaje). Proponga una frase más y haga lo
mismo.